In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [71]:
#Counting how many genes/onts are in each file
for f_path in sorted(glob.glob('../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/*.csv')):
    df = pd.read_csv(f_path)
    print(f_path)
    print(df['gene'].nunique(),'genes')
    print(df['annotation'].nunique(),'onts')
    

../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/vz_Brainmap_slice1_central_gene_ont.csv
572 genes
29 onts
../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/vz_Brainmap_slice1_peripheral_gene_ont.csv
572 genes
29 onts
../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/vz_Brainmap_slice1_punctate_gene_ont.csv
342 genes
29 onts
../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/vz_Brainmap_slice1_radial_gene_ont.csv
342 genes
29 onts
../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/vz_Brainmap_slice2_central_gene_ont.csv
560 genes
42 onts
../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/vz_Brainmap_slice2_peripheral_gene_ont.csv
560 genes
42 onts
../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/vz_Brainmap_slice2_punctate_gene_ont.csv
345 genes
42 onts
../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/vz_Brainmap_slice2_radial_gene_ont.csv
345 genes
42 onts
../../nf_pipeline/outputs/

In [24]:
metrics = ['peripheral','central','radial','punctate']
for metric in metrics:
    print(metric)
    f_paths = glob.glob('../../nf_pipeline/outputs/vz_Brainmap_20220602_direct/gene_ont/*{}*.csv'.format(metric))
    df = pd.concat((pd.read_csv(p) for p in f_paths))
    
    print('Num genes',df['gene'].nunique())
    print('Num ontologies',df['annotation'].nunique())
    fraction_positive = df['med_score'].gt(0).sum()/len(df)
    print('Fraction ann/gene groups with a positive score {:.3f}'.format(
        fraction_positive
    ))
    
    print('Num tot ann/gene groups',df.groupby(['annotation','gene']).ngroups)
    
    sig_df = df[
        df['bh_p'].le(0.05) &
        df['med_gene_spots'].ge(2)
    ]
    sig_df = sig_df.groupby(['annotation','gene']).filter(lambda g: g['experiment'].nunique() >= 3)
    
    print('Num sig genes',sig_df['gene'].nunique())
    print('Num sig ontologies',sig_df['annotation'].nunique())
    
    num_groups = sig_df.groupby(['annotation','gene']).ngroups
    print('Num sig ann/gene groups',num_groups)
    
    fraction_positive = sig_df['med_score'].gt(0).sum()/len(sig_df)
    print('Fraction sig ann/gene groups with a positive score {:.3f}'.format(
        fraction_positive
    ))
    
    num_groups_same_effect = sig_df.groupby(['annotation','gene'])['med_score'].apply(lambda v: all(v>0) or all(v<0)).sum()

    print('Number sig ann/gene groups with same direction of effect {}'.format(
        num_groups_same_effect
    ))
    
    print('Fraction sig ann/gene groups with same direction of effect {:.3f}'.format(
        num_groups_same_effect/num_groups
    ))
    
    print('There are Blank- MERFISH genes in the significant group',sig_df['gene'].str.contains('Blank-').any())
    print('')



peripheral
Num genes 589
Num ontologies 42
Fraction ann/gene groups with a positive score 0.410
Num tot ann/gene groups 16117
Num sig genes 112
Num sig ontologies 29
Num sig ann/gene groups 1682
Fraction sig ann/gene groups with a positive score 0.292
Number sig ann/gene groups with same direction of effect 1667
Fraction sig ann/gene groups with same direction of effect 0.991
There are Blank- MERFISH genes in the significant group False

central
Num genes 589
Num ontologies 42
Fraction ann/gene groups with a positive score 0.595
Num tot ann/gene groups 16117
Num sig genes 108
Num sig ontologies 29
Num sig ann/gene groups 1619
Fraction sig ann/gene groups with a positive score 0.727
Number sig ann/gene groups with same direction of effect 1610
Fraction sig ann/gene groups with same direction of effect 0.994
There are Blank- MERFISH genes in the significant group False

radial
Num genes 370
Num ontologies 42
Fraction ann/gene groups with a positive score 0.798
Num tot ann/gene groups 917